In [1]:
import tensorflow as tf

# Project imports
from components.load_step import load
from components.preprocess_step import preprocess
from components.train_test_split_step import tfds_train_test_split
from components.model_step import model



2024-05-08 15:03:27.294209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jcheung/.virtualenvs/scaled-ML-deployment/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:

data = [13.0, 10.6, 8.9, 7.9, 8.2, 7.8, 7.1, 7.1, 6.0, 6.2, 8.3, 7.3, 6.5, 5.5, 5.5, 5.3, 5.8]
print(sum(data))
print(len(data))
data1_mean = sum(data)/len(data)
data_2 = [36.6, 20.1, 18.8, 15.1, 12.0, 11.7, 10.9, 9.0, 11.9, 13.3, 8.8, 8.8, 6.1, 6.6, 6.5, 11.3, 11.2]
print(sum(data_2))
print(len(data_2))
data2_mean = sum(data_2)/len(data_2)


126.99999999999999
17
218.70000000000002
17


In [18]:
np.matmul(np.array(data) - data1_mean,np.array(data_2) - data2_mean)

200.2923529411765

In [11]:
import numpy as np
np.corrcoef(data, data_2)

array([[1.        , 0.86150232],
       [0.86150232, 1.        ]])

In [14]:
(36.6 + 20.1 + 18.8 + 15.1 + 12.0 + 11.7 + 10.9 + 9.0 + 11.9 + 13.3 + 8.8 + 8.8 + 6.1 + 6.6 + 6.5 + 11.3 + 11.2)/17

12.864705882352942

In [4]:
!pip show kfp

Name: kfp
Version: 1.8.19
Summary: KubeFlow Pipelines SDK
Home-page: https://github.com/kubeflow/pipelines
Author: The Kubeflow Authors
Author-email: 
License: UNKNOWN
Location: /Users/jcheung/.virtualenvs/scaled-ML-deployment/lib/python3.9/site-packages
Requires: absl-py, click, cloudpickle, Deprecated, docstring-parser, fire, google-api-core, google-api-python-client, google-auth, google-cloud-storage, jsonschema, kfp-pipeline-spec, kfp-server-api, kubernetes, protobuf, pydantic, PyYAML, requests-toolbelt, strip-hints, tabulate, typer, uritemplate
Required-by: 


In [3]:
data = load.mini_load_tensorflow_dataset(dataset_str='mnist',
                                        percent_of_dataset=10)
data = data.map(preprocess.preprocess_mnist_tfds,
                num_parallel_calls=tf.data.AUTOTUNE).batch(128)
train_dataset, test_dataset = tfds_train_test_split.train_test_split(data=data,
                                                                    train_proportion=.75)

# instantiate hyperparameters from arguments. This'll be changed for katib
hyperparameters = {'epochs': 5,
                'l1': 0.1,
                'l2': 0.1, 
                'num_hidden': 64,
                'learning_rate': 0.1}

MNIST = model.MNIST()
# with mlflow.start_run():
MNIST.fit_hp_search(xy_train=train_dataset,
                        xy_test=test_dataset,
                        hyperparameters=hyperparameters)
    # MLFlow Tracking parameters

2024-05-08 15:06:06.071334: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.



epoche 0:

epoche 1:
loss=9.974038124084473
accuracy=0.22477678954601288
val_loss=1.938489556312561
val_accuracy=0.2480263113975525

epoche 2:
loss=1.7430999279022217
accuracy=0.3299107253551483
val_loss=1.5612045526504517
val_accuracy=0.3835526406764984


KeyboardInterrupt: 